In [1]:
from pathlib import Path
from PIL import Image
import os

In [7]:
image = Image.open('honey_60.jpg')
def split_2x2_to_row(img):
    """将2x2网格图像切分并拼接成1行4列"""
    w, h = img.size
    half_w, half_h = w // 2, h // 2
    
    # 切分四个子图像 [左上, 右上, 左下, 右下]
    img1 = img.crop((0, 0, half_w, half_h))
    img2 = img.crop((half_w, 0, w, half_h))
    img3 = img.crop((0, half_h, half_w, h))
    img4 = img.crop((half_w, half_h, w, h))
    
    # 拼接成1行
    new_img = Image.new('RGB', (w * 2, half_h))
    new_img.paste(img1, (0, 0))
    new_img.paste(img2, (half_w, 0))
    new_img.paste(img3, (w, 0))
    new_img.paste(img4, (w + half_w, 0))
    
    return new_img
image = split_2x2_to_row(image)

In [8]:
image.save('0008_0019.png')

In [1]:
def compress_by_quality(input_path, output_path, quality=85):
    """通过降低质量压缩图像（推荐方法）
    
    Args:
        input_path: 输入文件路径
        output_path: 输出文件路径
        quality: JPEG质量 (1-100)，PNG会自动优化
    """
    with Image.open(input_path) as img:
        # 转换RGBA为RGB（JPG不支持透明度）
        if img.mode in ('RGBA', 'LA', 'P'):
            rgb_img = Image.new('RGB', img.size, (255, 255, 255))
            if img.mode == 'P':
                img = img.convert('RGBA')
            rgb_img.paste(img, mask=img.split()[-1] if img.mode in ('RGBA', 'LA') else None)
            img = rgb_img
        
        # 保存压缩后的图像
        if output_path.suffix.lower() in ['.jpg', '.jpeg']:
            img.save(output_path, 'JPEG', quality=quality, optimize=True)
        else:  # PNG
            img.save(output_path, 'PNG', optimize=True, compress_level=9)

In [5]:
def compress_by_resize(input_path, output_path, max_size=(1920, 1080), quality=85):
    """通过调整尺寸压缩图像（激进方法）
    
    Args:
        input_path: 输入文件路径
        output_path: 输出文件路径
        max_size: 最大尺寸 (width, height)
        quality: 保存质量
    """
    with Image.open(input_path) as img:
        # 保持宽高比缩放
        img.thumbnail(max_size, Image.Resampling.LANCZOS)
        
        if img.mode in ('RGBA', 'LA', 'P'):
            rgb_img = Image.new('RGB', img.size, (255, 255, 255))
            if img.mode == 'P':
                img = img.convert('RGBA')
            rgb_img.paste(img, mask=img.split()[-1] if img.mode in ('RGBA', 'LA') else None)
            img = rgb_img
        
        if output_path.suffix.lower() in ['.jpg', '.jpeg']:
            img.save(output_path, 'JPEG', quality=quality, optimize=True)
        else:
            img.save(output_path, 'PNG', optimize=True, compress_level=9)

In [6]:
current_dir = Path('.')
output_dir = Path('./compressed')
output_dir.mkdir(parents=True, exist_ok=True)

# 递归查找所有图像
image_extensions = {'.png', '.jpg', '.jpeg'}
image_files = [
    f for f in current_dir.rglob('*') 
    if f.is_file() and f.suffix.lower() in image_extensions
]
print(f"找到 {len(image_files)} 个图像文件")

找到 40 个图像文件


In [ ]:
image_files

In [11]:
# 压缩图像
for img_path in image_files:
    # 保持相对目录结构
    relative_path = img_path.relative_to(current_dir)
    output_path = output_dir / relative_path
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    original_size = img_path.stat().st_size
    
    # 方案1: 质量压缩（推荐）
    # compress_by_quality(img_path, output_path, quality=75)
    
    # 方案2: 尺寸+质量压缩（取消注释使用）
    compress_by_resize(img_path, output_path, max_size=(1920, 1080), quality=85)
    
    compressed_size = output_path.stat().st_size
    ratio = (1 - compressed_size / original_size) * 100
    
    print(f"{relative_path}: {original_size/1024:.1f}KB -> {compressed_size/1024:.1f}KB "
            f"(压缩 {ratio:.1f}%)")

static/images/method/dataset_pipeline_2.png: 848.0KB -> 694.3KB (压缩 18.1%)
static/images/method/RL_pipeline.png: 628.5KB -> 517.3KB (压缩 17.7%)
static/images/diff/page5/ours/0004_0017.png: 1288.9KB -> 998.7KB (压缩 22.5%)
static/images/diff/page5/ours/0004_0018.png: 861.7KB -> 752.4KB (压缩 12.7%)
static/images/diff/page5/seedream/0004_0017.png: 14518.3KB -> 1448.7KB (压缩 90.0%)
static/images/diff/page5/seedream/0004_0018.png: 13860.4KB -> 1346.3KB (压缩 90.3%)
static/images/diff/page5/AutoT2IS+gpt/0004_0017.png: 1402.9KB -> 1204.0KB (压缩 14.2%)
static/images/diff/page5/AutoT2IS+gpt/0004_0018.png: 1322.1KB -> 1121.1KB (压缩 15.2%)
static/images/diff/page2/ours/0017_0009.png: 953.4KB -> 820.1KB (压缩 14.0%)
static/images/diff/page2/ours/0017_0015.png: 1158.8KB -> 1007.2KB (压缩 13.1%)
static/images/diff/page2/seedream/0017_0009.png: 13240.0KB -> 766.1KB (压缩 94.2%)
static/images/diff/page2/seedream/0017_0015.png: 16399.4KB -> 1029.9KB (压缩 93.7%)
static/images/diff/page2/AutoT2IS/0017_0009.png: 998.1KB 